In [1]:
import os
exec(open(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.3
      /_/

Using Python version 3.7.1 (default, Dec 10 2018 22:54:23)
SparkSession available as 'spark'.


# Carregamento dos dados de treino

In [2]:
data = spark.read.csv("building_metadata.csv", header=True, inferSchema=True)

In [3]:
train = spark.read.csv("train.csv", header=True, inferSchema=True)

In [4]:
df_train = train.join(data, ['building_id'])

In [5]:
df_train.show(5)

+-----------+-----+-------------------+-------------+-------+-----------+-----------+----------+-----------+
|building_id|meter|          timestamp|meter_reading|site_id|primary_use|square_feet|year_built|floor_count|
+-----------+-----+-------------------+-------------+-------+-----------+-----------+----------+-----------+
|          0|    0|2016-01-01 00:00:00|          0.0|      0|  Education|     7432.0|      2008|       null|
|          1|    0|2016-01-01 00:00:00|          0.0|      0|  Education|     2720.0|      2004|       null|
|          2|    0|2016-01-01 00:00:00|          0.0|      0|  Education|     5376.0|      1991|       null|
|          3|    0|2016-01-01 00:00:00|          0.0|      0|  Education|    23685.0|      2002|       null|
|          4|    0|2016-01-01 00:00:00|          0.0|      0|  Education|   116607.0|      1975|       null|
+-----------+-----+-------------------+-------------+-------+-----------+-----------+----------+-----------+
only showing top 5 

In [5]:
weather_train = spark.read.csv("weather_train.csv", header=True, inferSchema=True)

In [23]:
weather_train.show(5)

+-------+-------------------+---------------+--------------+---------------+-----------------+------------------+--------------+----------+
|site_id|          timestamp|air_temperature|cloud_coverage|dew_temperature|precip_depth_1_hr|sea_level_pressure|wind_direction|wind_speed|
+-------+-------------------+---------------+--------------+---------------+-----------------+------------------+--------------+----------+
|      0|2016-01-01 00:00:00|           25.0|           6.0|           20.0|             null|            1019.7|           0.0|       0.0|
|      0|2016-01-01 01:00:00|           24.4|          null|           21.1|             -1.0|            1020.2|          70.0|       1.5|
|      0|2016-01-01 02:00:00|           22.8|           2.0|           21.1|              0.0|            1020.2|           0.0|       0.0|
|      0|2016-01-01 03:00:00|           21.1|           2.0|           20.6|              0.0|            1020.1|           0.0|       0.0|
|      0|2016-01-01 

In [6]:
data_train = df_train.join(weather_train, ['site_id', 'timestamp'])

In [8]:
data_train.show(5)

+-------+-------------------+-----------+-----+-------------+-----------+-----------+----------+-----------+---------------+--------------+---------------+-----------------+------------------+--------------+----------+
|site_id|          timestamp|building_id|meter|meter_reading|primary_use|square_feet|year_built|floor_count|air_temperature|cloud_coverage|dew_temperature|precip_depth_1_hr|sea_level_pressure|wind_direction|wind_speed|
+-------+-------------------+-----------+-----+-------------+-----------+-----------+----------+-----------+---------------+--------------+---------------+-----------------+------------------+--------------+----------+
|      0|2016-01-01 00:00:00|          0|    0|          0.0|  Education|     7432.0|      2008|       null|           25.0|           6.0|           20.0|             null|            1019.7|           0.0|       0.0|
|      0|2016-01-01 00:00:00|          1|    0|          0.0|  Education|     2720.0|      2004|       null|           25.0|

In [7]:
data_train.printSchema()

root
 |-- site_id: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- building_id: integer (nullable = true)
 |-- meter: integer (nullable = true)
 |-- meter_reading: double (nullable = true)
 |-- primary_use: string (nullable = true)
 |-- square_feet: double (nullable = true)
 |-- year_built: integer (nullable = true)
 |-- floor_count: double (nullable = true)
 |-- air_temperature: double (nullable = true)
 |-- cloud_coverage: double (nullable = true)
 |-- dew_temperature: double (nullable = true)
 |-- precip_depth_1_hr: double (nullable = true)
 |-- sea_level_pressure: double (nullable = true)
 |-- wind_direction: double (nullable = true)
 |-- wind_speed: double (nullable = true)



In [7]:
import pyspark.sql.functions as SF
from pyspark.sql.functions import udf
import pyspark.sql.types as ST
from pyspark.sql.functions import lit

In [11]:
data_train.filter(SF.col('meter_reading').isNull()).count()

0

# Explorando os dados de treino

In [10]:
# Verificando a quantidade de valores nulos nas colunas floor_count e precip_depth_1_hr

a = data_train.filter(SF.col('floor_count').isNull()).count()
b = data_train.filter(SF.col('precip_depth_1_hr').isNull()).count()

print (a, b)

16633726 3659684


Conclusão da célula acima ^ : dropar coluna 'floor_count' pois existem mais missing values do que dados, apesr de o tamanho do prédio ser um parâmetro de grande influência, de acordo com estudos

Tratar missing values da coluna precip_depth_1_hr com a média

In [8]:
# Dropando coluna 'floor_count' => muitos valores nulos

data_train = data_train.drop('floor_count')

In [9]:
# Substituindo missing values pela média da coluna

from pyspark.ml.feature import Imputer

imputer = Imputer(
    inputCols=['precip_depth_1_hr'], 
    outputCols=["precip_no_missing"]
)
data_train = imputer.fit(data_train).transform(data_train)

In [15]:
data_train.printSchema()

root
 |-- site_id: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- building_id: integer (nullable = true)
 |-- meter: integer (nullable = true)
 |-- meter_reading: double (nullable = true)
 |-- primary_use: string (nullable = true)
 |-- square_feet: double (nullable = true)
 |-- year_built: integer (nullable = true)
 |-- air_temperature: double (nullable = true)
 |-- cloud_coverage: double (nullable = true)
 |-- dew_temperature: double (nullable = true)
 |-- precip_depth_1_hr: double (nullable = true)
 |-- sea_level_pressure: double (nullable = true)
 |-- wind_direction: double (nullable = true)
 |-- wind_speed: double (nullable = true)
 |-- precip_no_missing: double (nullable = true)



In [16]:
#Verificando correlações entre algumas features e a variável alvo(meter reading)

corr1 = data_train.corr('square_feet', 'meter_reading')
corr2 = data_train.corr('floor_count', 'meter_reading')
corr3 = data_train.corr('air_temperature', 'meter_reading')
corr4 = data_train.corr('dew_temperature', 'meter_reading')

a = (corr1, corr2, corr3, corr4)

print(a)

(0.0243451719891813, -0.0034308352766083557, -0.004130718737031981, -0.003327480432342089)


In [19]:
#Agrupando o consumo de energia por tipo de geração {0: electricity, 1: chilledwater, 2: steam, 3: hotwater}

meter = data_train.groupBy(['meter']).agg(SF.sum('meter_reading')).alias('energy_type_consumption')
meter.show()

+-----+--------------------+
|meter|  sum(meter_reading)|
+-----+--------------------+
|    1|2.6469138369947457E9|
|    3|4.8424375217471313E8|
|    2|3.758367124840034E10|
|    0|2.0509979255654378E9|
+-----+--------------------+



In [12]:
# Média de consumo de energia por tipo de geração {0: electricity, 1: chilledwater, 2: steam, 3: hotwater}

meter = data_train.groupBy(['meter']).agg(SF.mean('meter_reading').alias('energy_type_avgconsumption'))
meter.show()

+-----+--------------------------+
|meter|energy_type_avgconsumption|
+-----+--------------------------+
|    1|         636.1287307376127|
|    3|         383.7977422611056|
|    2|        13985.626222980976|
|    0|        170.63067320028657|
+-----+--------------------------+



In [20]:
#Agrupando o consumo de energia por tipo de uso do prédio

primary_use_consumption = data_train.groupBy(['primary_use']).agg(SF.sum('meter_reading'))
primary_use_consumption.show()

+--------------------+--------------------+
|         primary_use|  sum(meter_reading)|
+--------------------+--------------------+
|            Services|  3.97066208180799E8|
|           Education|3.741569958853087E10|
|Entertainment/pub...| 1.070394479290633E9|
|             Parking| 3.621353672329998E7|
|Manufacturing/ind...| 3.544375192059997E7|
|Food sales and se...| 3.478257823299982E7|
|          Healthcare| 2.941853399864968E8|
|   Religious worship|   169620.2179000005|
|              Office|2.3108906238607793E9|
|  Technology/science|1.0557363489100015E7|
|     Public services|4.7844049827199876E8|
|               Other| 3.359877576220001E7|
|             Utility|2.8291403059499964E7|
| Lodging/residential| 5.982667954383855E8|
|              Retail|1.5746474144500503E7|
|   Warehouse/storage|   6079726.025100043|
+--------------------+--------------------+



In [21]:
# Quantidade de dados por tipo de uso primário

primary_use_count = data_train.groupBy(['primary_use']).agg(SF.count('meter_reading'))
primary_use_count.show()

+--------------------+--------------------+
|         primary_use|count(meter_reading)|
+--------------------+--------------------+
|            Services|               96510|
|           Education|             8126540|
|Entertainment/pub...|             2255388|
|             Parking|              213827|
|Manufacturing/ind...|              124488|
|Food sales and se...|              114066|
|          Healthcare|              398084|
|   Religious worship|               31783|
|              Office|             4380282|
|  Technology/science|               76731|
|     Public services|             1659241|
|               Other|              242219|
|             Utility|               55030|
| Lodging/residential|             2131455|
|              Retail|              112588|
|   Warehouse/storage|              111864|
+--------------------+--------------------+



In [11]:
# Média de consumo de energia por tipo de uso do prédio

primary_use_count = data_train.groupBy(['primary_use']).agg(SF.mean('meter_reading'))
primary_use_count.show()

+--------------------+------------------+
|         primary_use|avg(meter_reading)|
+--------------------+------------------+
|            Services| 4114.249385356948|
|           Education| 4604.136519174319|
|Entertainment/pub...|474.59438433237784|
|             Parking|169.35904597314644|
|Manufacturing/ind...| 284.7162129731377|
|Food sales and se...| 304.9337947591729|
|          Healthcare| 739.0031751753319|
|   Religious worship| 5.336822134474421|
|              Office| 527.5666324361717|
|  Technology/science|137.58928580495515|
|     Public services|  288.349009138515|
|               Other|138.71238739405254|
|             Utility| 514.1087235962196|
| Lodging/residential|280.68469446382187|
|              Retail|139.85925804260225|
|   Warehouse/storage| 54.34926361564081|
+--------------------+------------------+



In [20]:
# Anlisando os dados da coluna sea_level_pressure (desconfio que todos os dados são iguais)

data_train.describe('sea_level_pressure').show()

+-------+------------------+
|summary|sea_level_pressure|
+-------+------------------+
|  count|          18988655|
|   mean|1016.0857320604786|
| stddev| 7.060842611881579|
|    min|             968.2|
|    max|            1045.5|
+-------+------------------+



In [10]:
# Conclusão da célula acima: dropar a coluna sea_level_pressure pois a apmlitude de variação dos dados é muito baixa, provavelmente não causa tanta influência

# Drop da coluna sea_level_pressure

data_train = data_train.drop('sea_level_pressure')

In [9]:
# Identificando a quantidade de missing values em cada uma das colunas

columns = ['meter', 'primary_use', 'square_feet', 'year_built', 'air_temperature', 'cloud_coverage', 'dew_temperature', 'wind_direction', 'wind_speed']
missing_number = []

for count in columns:
    x = data_train.filter(SF.col(count).isNull()).count()
    missing_number.append(x)
    
print(missing_number)

[0, 0, 0, 12115911, 6163, 8735636, 9645, 1358553, 53181]


Conclusões da célula acima ^:

1. Somando os totais de valores nulos das colunas 'air_temperature', 'dew_temperature' e 'wind_speed' obtém-se um número de linhas com missing values que corresponde a, no máximo, 0.35% do tamanho da base. Por considerar esse valor irrisório, todas as linhas nulas dessas colunas vão ser dropadas


2. Como o percentual de missing values na coluna é muito alto, as colunas 'year_built' e 'cloud_coverage' serão dropadas. Estudos mostram que a idade de um prédio influencia o seu consumo, por esse motivo, considerou-se melhor não inputar tantos dados de forma aleatória (ver http://www.irbnet.de/daten/iconda/CIB_DC25108.pdf)


3. Substituir missing values pela média na coluna 'wind_direction'

In [11]:
# dropando as linhas que contêm valores nulos nas colunas 'air_temperature', 'dew_temperature', 'wind_speed'

data_train = data_train.na.drop(subset=['air_temperature', 'dew_temperature', 'wind_speed'])

In [12]:
# dropando as colunas 'year_built' e 'cloud_coverage'

data_train = data_train.drop('year_built', 'cloud_coverage')
data_train.printSchema()

root
 |-- site_id: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- building_id: integer (nullable = true)
 |-- meter: integer (nullable = true)
 |-- meter_reading: double (nullable = true)
 |-- primary_use: string (nullable = true)
 |-- square_feet: double (nullable = true)
 |-- air_temperature: double (nullable = true)
 |-- dew_temperature: double (nullable = true)
 |-- precip_depth_1_hr: double (nullable = true)
 |-- wind_direction: double (nullable = true)
 |-- wind_speed: double (nullable = true)
 |-- precip_no_missing: double (nullable = true)



In [13]:
# substituição de missing values usando o método imputer na coluna 'wind_direction'

imputer = Imputer(
    inputCols=['wind_direction'], 
    outputCols=["wind_direction_no_missing"]
)
data_train = imputer.fit(data_train).transform(data_train)

In [11]:
#Avaliando como a idade pode interferir no consumo de energia do prédio
data_train.groupBy(['year_built']).agg(SF.sum('meter_reading').alias('consumption')).sort(SF.col('consumption').desc()).show(10)

+----------+--------------------+
|year_built|         consumption|
+----------+--------------------+
|      null|4.042068313658111...|
|      2006| 1.661195065875976E8|
|      2009| 9.758516870349969E7|
|      1976|  8.67439377017974E7|
|      2007| 8.163683573850028E7|
|      1969| 7.244320173699997E7|
|      1930|     6.89204354895E7|
|      2010| 6.664581485770041E7|
|      1989| 6.620940334480016E7|
|      1952|6.4055442101500206E7|
+----------+--------------------+
only showing top 10 rows



In [12]:
#Avaliando as correlações entre as variáveis de tempo

corr1 = data_train.corr('wind_speed', 'dew_temperature')
corr2 = data_train.corr('dew_temperature', 'cloud_coverage')
corr3 = data_train.corr('air_temperature', 'wind_speed')
corr4 = data_train.corr('dew_temperature', 'cloud_coverage')
corr5 = data_train.corr('precip_depth_1_hr', 'cloud_coverage')

a = (corr1, corr2, corr3, corr4, corr5)

print(a)

(-0.1425580577090447, 0.06028352081244824, -0.08327778700727852, 0.06028352081244824, -0.010001658917668502)


# Data preparation - base de treino

In [20]:
data_train.printSchema()

root
 |-- site_id: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- building_id: integer (nullable = true)
 |-- meter: integer (nullable = true)
 |-- meter_reading: double (nullable = true)
 |-- primary_use: string (nullable = true)
 |-- square_feet: double (nullable = true)
 |-- air_temperature: double (nullable = true)
 |-- dew_temperature: double (nullable = true)
 |-- precip_depth_1_hr: double (nullable = true)
 |-- wind_direction: double (nullable = true)
 |-- wind_speed: double (nullable = true)
 |-- precip_no_missing: double (nullable = true)
 |-- wind_direction_no_missing: double (nullable = true)



In [14]:
# dropando as colunas que foram tratadas com o metodo inputer

data_train = data_train.drop('precip_depth_1_hr', 'wind_direction')

In [24]:
data_train.show(5)

+-------+-------------------+-----------+-----+-------------+-----------+-----------+---------------+---------------+----------+------------------+-------------------------+
|site_id|          timestamp|building_id|meter|meter_reading|primary_use|square_feet|air_temperature|dew_temperature|wind_speed| precip_no_missing|wind_direction_no_missing|
+-------+-------------------+-----------+-----+-------------+-----------+-----------+---------------+---------------+----------+------------------+-------------------------+
|      0|2016-01-01 00:00:00|          0|    0|          0.0|  Education|     7432.0|           25.0|           20.0|       0.0|0.7962514234616596|                      0.0|
|      0|2016-01-01 00:00:00|          1|    0|          0.0|  Education|     2720.0|           25.0|           20.0|       0.0|0.7962514234616596|                      0.0|
|      0|2016-01-01 00:00:00|          2|    0|          0.0|  Education|     5376.0|           25.0|           20.0|       0.0|0.

In [15]:
# transformando a coluna de tempo em timestamp

data_train = data_train.withColumn('timestamp_unix', SF.unix_timestamp('timestamp'))
data_train.printSchema()

root
 |-- site_id: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- building_id: integer (nullable = true)
 |-- meter: integer (nullable = true)
 |-- meter_reading: double (nullable = true)
 |-- primary_use: string (nullable = true)
 |-- square_feet: double (nullable = true)
 |-- air_temperature: double (nullable = true)
 |-- dew_temperature: double (nullable = true)
 |-- wind_speed: double (nullable = true)
 |-- precip_no_missing: double (nullable = true)
 |-- wind_direction_no_missing: double (nullable = true)
 |-- timestamp_unix: long (nullable = true)



In [16]:
# dropando a coluna que foi transformada

data_train = data_train.drop('timestamp')

In [17]:
# a coluna 'primary_use' precisa de um string index

from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="primary_use", outputCol="primary_use_index")
indexed_train = indexer.fit(data_train).transform(data_train)
indexed_train.printSchema()

root
 |-- site_id: integer (nullable = true)
 |-- building_id: integer (nullable = true)
 |-- meter: integer (nullable = true)
 |-- meter_reading: double (nullable = true)
 |-- primary_use: string (nullable = true)
 |-- square_feet: double (nullable = true)
 |-- air_temperature: double (nullable = true)
 |-- dew_temperature: double (nullable = true)
 |-- wind_speed: double (nullable = true)
 |-- precip_no_missing: double (nullable = true)
 |-- wind_direction_no_missing: double (nullable = true)
 |-- timestamp_unix: long (nullable = true)
 |-- primary_use_index: double (nullable = false)



In [18]:
# as colunas 'meter'e 'primary_use_index' precisam de um encode

from pyspark.ml.feature import OneHotEncoderEstimator

encoder = OneHotEncoderEstimator(inputCols=["meter", "primary_use_index"], outputCols=["meter_vec", "primary_use_vec"])
model = encoder.fit(indexed_train) # dataframe gerado com StringIndexer
encoded_train = model.transform(indexed_train)
encoded_train.printSchema()

root
 |-- site_id: integer (nullable = true)
 |-- building_id: integer (nullable = true)
 |-- meter: integer (nullable = true)
 |-- meter_reading: double (nullable = true)
 |-- primary_use: string (nullable = true)
 |-- square_feet: double (nullable = true)
 |-- air_temperature: double (nullable = true)
 |-- dew_temperature: double (nullable = true)
 |-- wind_speed: double (nullable = true)
 |-- precip_no_missing: double (nullable = true)
 |-- wind_direction_no_missing: double (nullable = true)
 |-- timestamp_unix: long (nullable = true)
 |-- primary_use_index: double (nullable = false)
 |-- meter_vec: vector (nullable = true)
 |-- primary_use_vec: vector (nullable = true)



In [19]:
# dropando as colunas que passaram pelo encoded

encoded_train = encoded_train.drop('meter', 'primary_use', 'primary_use_index')

In [20]:
encoded_train.printSchema()

root
 |-- site_id: integer (nullable = true)
 |-- building_id: integer (nullable = true)
 |-- meter_reading: double (nullable = true)
 |-- square_feet: double (nullable = true)
 |-- air_temperature: double (nullable = true)
 |-- dew_temperature: double (nullable = true)
 |-- wind_speed: double (nullable = true)
 |-- precip_no_missing: double (nullable = true)
 |-- wind_direction_no_missing: double (nullable = true)
 |-- timestamp_unix: long (nullable = true)
 |-- meter_vec: vector (nullable = true)
 |-- primary_use_vec: vector (nullable = true)



In [21]:
# 'site_id' e 'building_id' não servem em nada para o modelo

encoded_train = encoded_train.drop('site_id', 'building_id')


In [22]:
encoded_train.printSchema()

root
 |-- meter_reading: double (nullable = true)
 |-- square_feet: double (nullable = true)
 |-- air_temperature: double (nullable = true)
 |-- dew_temperature: double (nullable = true)
 |-- wind_speed: double (nullable = true)
 |-- precip_no_missing: double (nullable = true)
 |-- wind_direction_no_missing: double (nullable = true)
 |-- timestamp_unix: long (nullable = true)
 |-- meter_vec: vector (nullable = true)
 |-- primary_use_vec: vector (nullable = true)



In [23]:
# criando o vetor de features

from pyspark.ml.feature import VectorAssembler

vectorAssembler = VectorAssembler(inputCols = ['square_feet', 'air_temperature', 'dew_temperature', 'wind_speed', 'precip_no_missing', 'wind_direction_no_missing', 'timestamp_unix', 'meter_vec', 'primary_use_vec'], outputCol = 'features')
train_base = vectorAssembler.transform(encoded_train)
train_base = train_base.select(['features', 'meter_reading'])
train_base.show(3)

+--------------------+-------------+
|            features|meter_reading|
+--------------------+-------------+
|(25,[0,1,2,4,6,7,...|          0.0|
|(25,[0,1,2,4,6,7,...|          0.0|
|(25,[0,1,2,4,6,7,...|          0.0|
+--------------------+-------------+
only showing top 3 rows



In [44]:
train_base = train_base.withColumnRenamed('meter_reading', 'label')
train_base.show(3)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(25,[0,1,2,4,6,7,...|  0.0|
|(25,[0,1,2,4,6,7,...|  0.0|
|(25,[0,1,2,4,6,7,...|  0.0|
+--------------------+-----+
only showing top 3 rows



# Data preparation - base de testes

In [25]:
df_test = spark.read.csv("test.csv", header=True, inferSchema=True)

In [26]:
df_test = df_test.join(data, ['building_id'])

In [27]:
weather_test = spark.read.csv('weather_test.csv', header=True, inferSchema=True)

In [28]:
data_test = df_test.join(weather_test, ['site_id', 'timestamp'])

In [29]:
# dropando as colunas que foram dropadas na base de treino

data_test = data_test.drop('floor_count', 'sea_level_pressure', 'year_built', 'cloud_coverage')

In [30]:
data_test.printSchema()

root
 |-- site_id: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- building_id: integer (nullable = true)
 |-- row_id: integer (nullable = true)
 |-- meter: integer (nullable = true)
 |-- primary_use: string (nullable = true)
 |-- square_feet: double (nullable = true)
 |-- air_temperature: double (nullable = true)
 |-- dew_temperature: double (nullable = true)
 |-- precip_depth_1_hr: double (nullable = true)
 |-- wind_direction: double (nullable = true)
 |-- wind_speed: double (nullable = true)



In [11]:
# Identificando a quantidade de missing values em cada uma das colunas

columns = ['meter', 'primary_use', 'square_feet', 'air_temperature', 'dew_temperature', 'precip_depth_1_hr', 'wind_direction', 'wind_speed']
missing_number = []

for count in columns:
    x = data_test.filter(SF.col(count).isNull()).count()
    missing_number.append(x)
    
print(missing_number)

[0, 0, 0, 22872, 61770, 7604260, 2780535, 103060]


Conclusão da célula acima:

1. Aplicar o método Imputer para 'preecip_depth_1_hr' e 'wind_direction'

2. Dropar as linhas nulas de 'air_temperature', 'dew_temperature' e 'wind_speed' 

In [31]:
# substituição de missing values usando o método imputer

imputer = Imputer(
    inputCols=['precip_depth_1_hr', 'wind_direction'], 
    outputCols=['precip_no_missing','wind_direction_no_missing']
)
data_test = imputer.fit(data_test).transform(data_test)

In [32]:
# dropando as linhas que contêm valores nulos nas colunas 'air_temperature', 'dew_temperature', 'wind_speed'

data_test = data_test.na.drop(subset=['air_temperature', 'dew_temperature', 'wind_speed'])

In [33]:
data_test.printSchema()

root
 |-- site_id: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- building_id: integer (nullable = true)
 |-- row_id: integer (nullable = true)
 |-- meter: integer (nullable = true)
 |-- primary_use: string (nullable = true)
 |-- square_feet: double (nullable = true)
 |-- air_temperature: double (nullable = true)
 |-- dew_temperature: double (nullable = true)
 |-- precip_depth_1_hr: double (nullable = true)
 |-- wind_direction: double (nullable = true)
 |-- wind_speed: double (nullable = true)
 |-- precip_no_missing: double (nullable = true)
 |-- wind_direction_no_missing: double (nullable = true)



In [34]:
# a coluna 'primary_use' precisa de um string index

indxr = StringIndexer(inputCol="primary_use", outputCol="primary_use_index")
indexed_test = indxr.fit(data_test).transform(data_test)
indexed_test.printSchema()

root
 |-- site_id: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- building_id: integer (nullable = true)
 |-- row_id: integer (nullable = true)
 |-- meter: integer (nullable = true)
 |-- primary_use: string (nullable = true)
 |-- square_feet: double (nullable = true)
 |-- air_temperature: double (nullable = true)
 |-- dew_temperature: double (nullable = true)
 |-- precip_depth_1_hr: double (nullable = true)
 |-- wind_direction: double (nullable = true)
 |-- wind_speed: double (nullable = true)
 |-- precip_no_missing: double (nullable = true)
 |-- wind_direction_no_missing: double (nullable = true)
 |-- primary_use_index: double (nullable = false)



In [35]:
# as colunas 'meter'e 'primary_use_index' precisam de um encode

encdr = OneHotEncoderEstimator(inputCols=["meter", "primary_use_index"], outputCols=["meter_vec", "primary_use_vec"])
model_test = encdr.fit(indexed_test) # dataframe gerado com StringIndexer
encoded_test = model.transform(indexed_test)
encoded_test.printSchema()

root
 |-- site_id: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- building_id: integer (nullable = true)
 |-- row_id: integer (nullable = true)
 |-- meter: integer (nullable = true)
 |-- primary_use: string (nullable = true)
 |-- square_feet: double (nullable = true)
 |-- air_temperature: double (nullable = true)
 |-- dew_temperature: double (nullable = true)
 |-- precip_depth_1_hr: double (nullable = true)
 |-- wind_direction: double (nullable = true)
 |-- wind_speed: double (nullable = true)
 |-- precip_no_missing: double (nullable = true)
 |-- wind_direction_no_missing: double (nullable = true)
 |-- primary_use_index: double (nullable = false)
 |-- meter_vec: vector (nullable = true)
 |-- primary_use_vec: vector (nullable = true)



In [36]:
# dropando as colunas que passaram pelo encoded

encoded_test = encoded_test.drop('meter', 'primary_use', 'primary_use_index')
encoded_test.printSchema()

root
 |-- site_id: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- building_id: integer (nullable = true)
 |-- row_id: integer (nullable = true)
 |-- square_feet: double (nullable = true)
 |-- air_temperature: double (nullable = true)
 |-- dew_temperature: double (nullable = true)
 |-- precip_depth_1_hr: double (nullable = true)
 |-- wind_direction: double (nullable = true)
 |-- wind_speed: double (nullable = true)
 |-- precip_no_missing: double (nullable = true)
 |-- wind_direction_no_missing: double (nullable = true)
 |-- meter_vec: vector (nullable = true)
 |-- primary_use_vec: vector (nullable = true)



In [37]:
# dropando as colunas que foram tratadas com o metodo inputer

encoded_test = encoded_test.drop('precip_depth_1_hr', 'wind_direction')

In [38]:
# transformando a coluna de tempo em timestamp

encoded_test = encoded_test.withColumn('timestamp_unix', SF.unix_timestamp('timestamp'))

# dropando a coluna que foi transformada

encoded_test = encoded_test.drop('timestamp')

encoded_test.printSchema()

root
 |-- site_id: integer (nullable = true)
 |-- building_id: integer (nullable = true)
 |-- row_id: integer (nullable = true)
 |-- square_feet: double (nullable = true)
 |-- air_temperature: double (nullable = true)
 |-- dew_temperature: double (nullable = true)
 |-- wind_speed: double (nullable = true)
 |-- precip_no_missing: double (nullable = true)
 |-- wind_direction_no_missing: double (nullable = true)
 |-- meter_vec: vector (nullable = true)
 |-- primary_use_vec: vector (nullable = true)
 |-- timestamp_unix: long (nullable = true)



In [39]:
# as colunas 'site_id', 'building_id' e 'row_id' não servem em nada para o modelo

encoded_test = encoded_test.drop('site_id', 'building_id', 'row_id')
encoded_test.printSchema()

root
 |-- square_feet: double (nullable = true)
 |-- air_temperature: double (nullable = true)
 |-- dew_temperature: double (nullable = true)
 |-- wind_speed: double (nullable = true)
 |-- precip_no_missing: double (nullable = true)
 |-- wind_direction_no_missing: double (nullable = true)
 |-- meter_vec: vector (nullable = true)
 |-- primary_use_vec: vector (nullable = true)
 |-- timestamp_unix: long (nullable = true)



In [40]:
# reordenando as colunas para ficar na mesma ordem do dataframe de treino

encoded_test = encoded_test.select('square_feet', 'air_temperature', 'dew_temperature', 'wind_speed', 'precip_no_missing', 'wind_direction_no_missing', 'timestamp_unix', 'meter_vec', 'primary_use_vec')
encoded_test.printSchema()

root
 |-- square_feet: double (nullable = true)
 |-- air_temperature: double (nullable = true)
 |-- dew_temperature: double (nullable = true)
 |-- wind_speed: double (nullable = true)
 |-- precip_no_missing: double (nullable = true)
 |-- wind_direction_no_missing: double (nullable = true)
 |-- timestamp_unix: long (nullable = true)
 |-- meter_vec: vector (nullable = true)
 |-- primary_use_vec: vector (nullable = true)



In [41]:
# criando o vetor de features

vectorAssembler = VectorAssembler(inputCols = ['square_feet', 'air_temperature', 'dew_temperature', 'wind_speed', 'precip_no_missing', 'wind_direction_no_missing', 'timestamp_unix', 'meter_vec', 'primary_use_vec'], outputCol = 'features')
test_base = vectorAssembler.transform(encoded_test)
test_base = test_base.select(['features'])
test_base.show(3)

+--------------------+
|            features|
+--------------------+
|(25,[0,1,2,3,5,6,...|
|(25,[0,1,2,3,5,6,...|
|(25,[0,1,2,3,5,6,...|
+--------------------+
only showing top 3 rows



# Criando o modelo de regressão múltipla

In [45]:
from pyspark.ml.regression import LinearRegression

# Cria o modelo
linearRegression = LinearRegression(featuresCol='features', labelCol='label')

# Treina o modelo com a base de treinamento (data frame formatado)
lrModel = linearRegression.fit(train_base)

# Aplica o modelo treinado para predizer a saída (label) na base de testes
predictions = lrModel.transform(test_base)

In [47]:
predictions.show(20)

+--------------------+-------------------+
|            features|         prediction|
+--------------------+-------------------+
|(25,[0,1,2,3,5,6,...|-2072.7602130970918|
|(25,[0,1,2,3,5,6,...|-2205.1804752209573|
|(25,[0,1,2,3,5,6,...|-2130.5395125297364|
|(25,[0,1,2,3,5,6,...|-1616.0058581948688|
|(25,[0,1,2,3,5,6,...|  995.3599306579272|
|(25,[0,1,2,3,5,6,...|-2056.7978385456663|
|(25,[0,1,2,3,5,6,...| -5601.790773884102|
|(25,[0,1,2,3,5,6,...| 1120.8950136178755|
|(25,[0,1,2,3,5,6,...|  793.9762024768861|
|(25,[0,1,2,3,5,6,...| -572.7185468437383|
|(25,[0,1,2,3,5,6,...|  -5305.65946889497|
|(25,[0,1,2,3,5,6,...| -5632.578280035959|
|(25,[0,1,2,3,5,6,...|  4741.997022825672|
|(25,[0,1,2,3,5,6,...|  -902.532680181117|
|(25,[0,1,2,3,5,6,...| -5343.975942660763|
|(25,[0,1,2,3,5,6,...|  511.2334757326462|
|(25,[0,1,2,3,5,6,...| 184.31466459165676|
|(25,[0,1,2,3,5,6,...|  142.2440780912002|
|(25,[0,1,2,3,5,6,...| -184.6747330497892|
|(25,[0,1,2,3,5,6,...|-3705.0098785322043|
+----------

In [48]:
# avaliando o erro médio quadrático e o r2

trainingSummary = lrModel.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 153637.048438
r2: 0.001636


In [ ]:
train_base.printSchema()